In [ ]:
import numpy as np
import os
import torch
import torchvision
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import PIL.Image as Image
import time
import copy

import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 64
img_size = 320
img_crop = 300
epochs = 10


# 1. Create Data transforms and Data loaders:

In [ ]:
data_transforms_train = transforms.Compose([
        transforms.Resize((img_size,img_size)),
        transforms.CenterCrop(img_crop),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
data_transforms_test = transforms.Compose([
        transforms.Resize((img_size,img_size)),
        transforms.CenterCrop(img_crop),

        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

data_transforms_val = transforms.Compose([
        transforms.Resize((img_size,img_size)),
        transforms.CenterCrop(img_crop),

        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


path = '../input/birds-augmented-dataset/AugmentedDataset/'


train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(path+'train', transform=data_transforms_train),
    batch_size=batch_size, shuffle=True, num_workers=4)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(path+'valid',transform=data_transforms_val),
    batch_size=200, shuffle=True, num_workers=4)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(path+'test',transform=data_transforms_test),
    batch_size=1, shuffle=False, num_workers=1)



dataloaders = {"train":train_loader, "valid":val_loader}

# 2. Training and evaluation function:

In [ ]:
def train(model, epoch):
    model.train()
    n_batches = 0
    acc_train = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        #forward
        preds= model(data)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        m = nn.Softmax(dim=1)
        probs = m(preds)
        preds_classes = probs.max(1, keepdim=True)[1]
        if batch_idx % 25 == 0:
            print('[{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
        
        acc_train += torch.sum(torch.squeeze(preds_classes)==labels).item()/labels.shape[0]
        n_batches += 1
        
    acc_train /= n_batches
    print('Epoch = {} ... Train Accuracy = {:.2f}'.format(epoch,100*acc_train))
    
        


def validation(model):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        
        for data, labels in val_loader:
            data, labels = data.to(device), labels.to(device)
            preds = model(data)
            # sum up batch loss
            validation_loss += criterion(preds, labels).data.item()
            m = nn.Softmax(dim=1)
            probs = m(preds)
            preds_classes = probs.max(1, keepdim=True)[1]
            correct += preds_classes.eq(labels.data.view_as(preds_classes)).sum()
        validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    
    return preds_classes,labels

# Model:

In [ ]:
model = torchvision.models.resnet152(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
for param in model.layer4.parameters():
    param.requires_grad = True
    

model.fc=nn.Sequential(
               nn.Dropout(p=0.4),
               nn.Linear(model.fc.in_features, 1000),
               nn.Dropout(p=0.2),
               nn.ReLU(inplace=True),
               nn.Linear(1000, 20))
      
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) 
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.75)

In [ ]:
epochs=20
for epoch in range(1, epochs + 1):
    print("################################################# EPOCH", epoch)
    train(model, epoch)
    preds = validation(model)
    model_file = 'experiments' + '/model_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)
    lr_scheduler.step()

In [ ]:
save = False
if save:
    torch.save(model.state_dict(), '/kaggle/working/model152.pt')

# Ensemble the models:

In [ ]:
#Change this model by another one.
model1 = torchvision.models.resnet152(pretrained=True)

for param in model1.parameters():
    param.requires_grad = False
    
for param in model1.layer4.parameters():
    param.requires_grad = True
    
model1.fc = nn.Sequential(
               nn.Dropout(p=0.4),
               nn.Linear(model1.fc.in_features, 1000),
               nn.Dropout(p=0.2),
               nn.ReLU(inplace=True),
               nn.Linear(1000, 20))

model1.load_state_dict(torch.load('../input/predictions/model_resnet101_orig_300_64.pt'),strict=False)

model1.to('cuda')
model1.eval()
print('model 1 loaded')

In [ ]:
preds1,labels = validation(model1)

In [ ]:
#Change this model by another one.
model2 = torchvision.models.resnext101_32x8d(pretrained=True)

for param in model1.parameters():
    param.requires_grad = False
    
for param in model1.layer4.parameters():
    param.requires_grad = True
    
model2.fc = nn.Sequential(
               nn.Dropout(p=0.4),
               nn.Linear(model2.fc.in_features, 1000),
               nn.Dropout(p=0.2),
               nn.ReLU(inplace=True),
               nn.Linear(1000, 20))

model2.load_state_dict(torch.load('/kaggle/input/predictions/model_resnext101_32x8d_300_64.pt'),strict=False)

model2.to('cuda')
model2.eval()
print('model 2 loaded')

In [ ]:
preds2,labels = validation(model2)

# Testing function to compute the predictions of each model based on duplicate testing and compute their confidence.

In [ ]:
def testing(model):
    
    test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(path+'test/',transform=data_transforms_test),
    batch_size=1, shuffle=False, num_workers=1)

    preds = np.array([])
    probas_ = []
    model.eval()
    with torch.no_grad():
        for i, (data, labels) in tqdm(enumerate(test_loader, 0)):
            data, labels = data.to(device), labels.to(device)
            output1 = model(data)

            sm = nn.Softmax(dim=1)(output1)
            pred = sm.max(1, keepdim=True)[1]    
            preds = np.hstack((preds, torch.squeeze(pred).cpu().numpy()))
            probas_.append(sm.max().item())

    p_=[]

    for i in range(7):
        if i in [2,3,4,5]:
            p_.append(probas_[i*517:(i+1)*517])

    p_ = np.transpose(np.array(p_))
    max_ = p_.argmax(axis=1)
    max_probs = p_.max(axis=1)

    preds_=[]

    for i in range(7):
        if i in [2,3,4,5]:
            preds_.append(preds[i*517:(i+1)*517])


    preds_ = np.transpose(np.array(preds_))
    preds_final = []
    for i in range(len(max_)):

        preds_final.append(int(preds_[i][max_[i]]))

    return preds_final,max_probs

# Perform Duplicate Testing for each model:

In [ ]:
preds_final1,max_probs1 = testing(model1)
preds_final2,max_probs2 = testing(model2)

# Confidence based predictions:

**Use the prediction of the model that has more confidence score to the test image**

In [ ]:
submission=[]
first_model = 0
second_model = 0
for i in range(len(preds_final1)):
    if max_probs1[i] >= max_probs2[i]:
        submission.append(preds_final1[i])
        first_model +=1
        
    else:
        submission.append(preds_final2[i])
        second_model += 1

In [ ]:
print(f'The final submission took {first_model} of First model and {second_model} of Second model')

# Save Submissions:

In [ ]:
def save_subs(idx,preds):
    f = open("submission"+str(idx)+".csv", "w")
    f.write("Id,Category\n")
    for (n,_),p in zip(test_loader.dataset.samples,preds):
        f.write("{},{}\n".format(n.split('/')[-1].split('.')[0], int(p)))
    f.close()

In [ ]:
save_subs('F7',preds)